In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import numpy as np

class AIModelMonitor:
    """
    Système de monitoring et amélioration continue pour le modèle de recommandations IA
    Surveille les performances, calcule les métriques et déclenche le réentraînement si nécessaire
    """
    
    def __init__(self, api_base_url='http://localhost:8080/api'):
        self.api_base_url = api_base_url
        self.token = None
        self.metrics_history = []
    
    def login(self, email, password):
        """Authentification à l'API"""
        try:
            response = requests.post(
                f"{self.api_base_url}/v1/auth/login",
                json={'email': email, 'motDePasse': password}
            )
            
            if response.status_code == 200:
                self.token = response.json()['token']
                print("✅ Authentifié pour le monitoring")
                return True
            return False
        except Exception as e:
            print(f"❌ Erreur authentification: {e}")
            return False
    
    def monitor_recommendation_quality(self):
        """
        Surveille la qualité des recommandations
        Calcule les métriques clés : CTR, Conversion Rate, Diversity, Coverage
        """
        print("\n📊 MONITORING DE LA QUALITÉ DES RECOMMANDATIONS")
        print("=" * 80)
        
        headers = {'Authorization': f'Bearer {self.token}'}
        
        try:
            # Récupérer les données des 7 derniers jours
            interactions = self._get_recent_interactions(headers, days=7)
            recommendations = self._get_recommendation_logs(headers, days=7)
            
            if not interactions or not recommendations:
                print("⚠️ Pas assez de données pour calculer les métriques")
                return None
            
            # Calculer les métriques
            metrics = {
                'ctr': self._calculate_ctr(interactions, recommendations),
                'conversion_rate': self._calculate_conversion_rate(interactions),
                'diversity_score': self._calculate_diversity(recommendations),
                'coverage': self._calculate_coverage(recommendations),
                'avg_rating': self._calculate_avg_rating(headers),
                'response_time': self._calculate_avg_response_time(recommendations),
                'timestamp': datetime.now().isoformat()
            }
            
            # Afficher les résultats
            print(f"\n📈 MÉTRIQUES (7 derniers jours):")
            print(f"   CTR (Click-Through Rate): {metrics['ctr']:.2%}")
            print(f"   Taux de conversion (Favoris): {metrics['conversion_rate']:.2%}")
            print(f"   Score de diversité: {metrics['diversity_score']:.2f}/1.0")
            print(f"   Couverture du catalogue: {metrics['coverage']:.2%}")
            print(f"   Note moyenne: {metrics['avg_rating']:.2f}/5.0")
            print(f"   Temps de réponse moyen: {metrics['response_time']:.0f}ms")
            
            # Évaluer la santé du système
            self._evaluate_system_health(metrics)
            
            # Sauvegarder l'historique
            self.metrics_history.append(metrics)
            self._save_metrics_history()
            
            return metrics
            
        except Exception as e:
            print(f"❌ Erreur lors du monitoring: {e}")
            return None
    
    def _get_recent_interactions(self, headers, days=7):
        """Récupère les interactions récentes"""
        try:
            response = requests.get(
                f"{self.api_base_url}/v1/recette-interactions/all",
                headers=headers,
                timeout=30
            )
            
            if response.status_code == 200:
                interactions = response.json()
                
                # Filtrer par date
                cutoff_date = datetime.now() - timedelta(days=days)
                recent = [
                    i for i in interactions
                    if datetime.fromisoformat(i['dateInteraction'].replace('Z', '+00:00')) > cutoff_date
                ]
                
                return recent
            return []
        except Exception as e:
            print(f"   ⚠️ Erreur récupération interactions: {e}")
            return []
    
    def _get_recommendation_logs(self, headers, days=7):
        """Récupère les logs de recommandations (si disponible dans votre système)"""
        # Si vous avez un endpoint pour les logs de recommandations
        # Sinon, on peut simuler ou utiliser les interactions de type CONSULTATION
        try:
            # Utiliser les consultations comme proxy
            interactions = self._get_recent_interactions(headers, days)
            consultations = [i for i in interactions if i['typeInteraction'] == 'CONSULTATION']
            return consultations
        except Exception as e:
            print(f"   ⚠️ Erreur récupération logs: {e}")
            return []
    
    def _calculate_ctr(self, interactions, recommendations):
        """Calcule le Click-Through Rate"""
        if not recommendations:
            return 0.0
        
        # CTR = (Clics / Impressions) * 100
        impressions = len(recommendations)
        clicks = len([i for i in interactions if i['typeInteraction'] == 'CONSULTATION'])
        
        return clicks / impressions if impressions > 0 else 0.0
    
    def _calculate_conversion_rate(self, interactions):
        """Calcule le taux de conversion (ajout aux favoris)"""
        if not interactions:
            return 0.0
        
        # Conversion = (Favoris / Total Interactions) * 100
        total = len(interactions)
        conversions = len([i for i in interactions if i['typeInteraction'] == 'FAVORI_AJOUTE'])
        
        return conversions / total if total > 0 else 0.0
    
    def _calculate_diversity(self, recommendations):
        """Calcule le score de diversité des recommandations"""
        if not recommendations:
            return 0.0
        
        # Diversité basée sur le nombre de catégories différentes
        categories = set()
        
        for rec in recommendations:
            recipe = rec.get('recetteEntity', {})
            categories.add(recipe.get('typeRecette', 'unknown'))
        
        # Score normalisé (on suppose 3 types principaux: entrée, plat, dessert)
        max_categories = 3
        diversity = len(categories) / max_categories
        
        return min(diversity, 1.0)
    
    def _calculate_coverage(self, recommendations):
        """Calcule la couverture du catalogue"""
        headers = {'Authorization': f'Bearer {self.token}'}
        
        try:
            # Récupérer toutes les recettes
            response = requests.get(
                f"{self.api_base_url}/v1/recettes/all",
                headers=headers,
                timeout=30
            )
            
            if response.status_code == 200:
                all_recipes = response.json()
                total_recipes = len(all_recipes)
                
                # Recettes recommandées
                recommended_ids = set()
                for rec in recommendations:
                    recommended_ids.add(rec.get('recetteId'))
                
                coverage = len(recommended_ids) / total_recipes if total_recipes > 0 else 0.0
                return coverage
        except:
            pass
        
        return 0.0
    
    def _calculate_avg_rating(self, headers):
        """Calcule la note moyenne des recettes recommandées"""
        try:
            response = requests.get(
                f"{self.api_base_url}/v1/notes/all",
                headers=headers,
                timeout=30
            )
            
            if response.status_code == 200:
                ratings = response.json()
                
                if ratings:
                    avg = sum(r['valeur'] for r in ratings) / len(ratings)
                    return avg
        except:
            pass
        
        return 0.0
    
    def _calculate_avg_response_time(self, recommendations):
        """Calcule le temps de réponse moyen (simulé)"""
        # Dans un vrai système, vous mesureriez le temps de réponse de l'API
        # Ici on simule une valeur raisonnable
        return np.random.normal(500, 100)  # ~500ms
    
    def _evaluate_system_health(self, metrics):
        """Évalue la santé globale du système et donne des alertes"""
        print(f"\n🏥 ÉVALUATION DE LA SANTÉ DU SYSTÈME:")
        
        issues = []
        warnings = []
        
        # Vérifier le CTR
        if metrics['ctr'] < 0.05:  # Moins de 5%
            issues.append("❌ CTR très faible - Les recommandations ne sont pas cliquées")
        elif metrics['ctr'] < 0.10:
            warnings.append("⚠️ CTR faible - Amélioration possible")
        
        # Vérifier le taux de conversion
        if metrics['conversion_rate'] < 0.05:
            issues.append("❌ Taux de conversion très faible - Les recettes ne sont pas sauvegardées")
        elif metrics['conversion_rate'] < 0.15:
            warnings.append("⚠️ Taux de conversion moyen - Optimisation recommandée")
        
        # Vérifier la diversité
        if metrics['diversity_score'] < 0.3:
            issues.append("❌ Diversité très faible - Recommandations trop homogènes")
        elif metrics['diversity_score'] < 0.5:
            warnings.append("⚠️ Diversité moyenne - Augmenter la variété")
        
        # Vérifier la couverture
        if metrics['coverage'] < 0.20:
            warnings.append("⚠️ Couverture faible - Beaucoup de recettes jamais recommandées")
        
        # Vérifier les notes
        if metrics['avg_rating'] < 3.0:
            issues.append("❌ Notes très basses - Qualité des recommandations à revoir")
        elif metrics['avg_rating'] < 3.5:
            warnings.append("⚠️ Notes moyennes - Amélioration souhaitée")
        
        # Afficher les résultats
        if not issues and not warnings:
            print("   ✅ Système en bonne santé - Toutes les métriques sont bonnes")
        else:
            if issues:
                print("   PROBLÈMES CRITIQUES:")
                for issue in issues:
                    print(f"      {issue}")
            
            if warnings:
                print("   AVERTISSEMENTS:")
                for warning in warnings:
                    print(f"      {warning}")
        
        # Recommandation d'action
        if len(issues) >= 2:
            print("\n   🔄 RECOMMANDATION: Réentraînement du modèle nécessaire")
            return 'CRITICAL'
        elif len(warnings) >= 2:
            print("\n   💡 RECOMMANDATION: Optimisation suggérée")
            return 'WARNING'
        else:
            print("\n   ✓ RECOMMANDATION: Continuer la surveillance")
            return 'HEALTHY'
    
    def _save_metrics_history(self):
        """Sauvegarde l'historique des métriques"""
        try:
            with open('metrics_history.json', 'w') as f:
                json.dump(self.metrics_history, f, indent=2)
            print(f"\n💾 Métriques sauvegardées (total: {len(self.metrics_history)} entrées)")
        except Exception as e:
            print(f"⚠️ Erreur sauvegarde métriques: {e}")
    
    def generate_performance_report(self):
        """Génère un rapport de performance détaillé"""
        print("\n📋 GÉNÉRATION DU RAPPORT DE PERFORMANCE")
        print("=" * 80)
        
        if not self.metrics_history:
            print("⚠️ Pas de données historiques disponibles")
            return
        
        df = pd.DataFrame(self.metrics_history)
        
        # Créer le rapport
        report_path = 'performance_report.txt'
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("RAPPORT DE PERFORMANCE - SYSTÈME DE RECOMMANDATIONS IA\n")
            f.write("=" * 80 + "\n\n")
            f.write(f"Date de génération: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Période d'analyse: {len(self.metrics_history)} mesures\n\n")
            
            # Statistiques globales
            f.write("1. STATISTIQUES GLOBALES\n")
            f.write("-" * 80 + "\n")
            
            for metric in ['ctr', 'conversion_rate', 'diversity_score', 'coverage', 'avg_rating']:
                values = df[metric].dropna()
                if len(values) > 0:
                    f.write(f"\n{metric.upper().replace('_', ' ')}:\n")
                    f.write(f"   Moyenne: {values.mean():.4f}\n")
                    f.write(f"   Médiane: {values.median():.4f}\n")
                    f.write(f"   Min: {values.min():.4f}\n")
                    f.write(f"   Max: {values.max():.4f}\n")
                    f.write(f"   Écart-type: {values.std():.4f}\n")
            
            # Évolution
            f.write("\n2. ÉVOLUTION DES MÉTRIQUES\n")
            f.write("-" * 80 + "\n")
            
            if len(df) >= 2:
                for metric in ['ctr', 'conversion_rate', 'diversity_score']:
                    first_value = df[metric].iloc[0]
                    last_value = df[metric].iloc[-1]
                    change = ((last_value - first_value) / first_value) * 100 if first_value != 0 else 0
                    
                    trend = "↗️ Amélioration" if change > 5 else "↘️ Dégradation" if change < -5 else "→ Stable"
                    
                    f.write(f"\n{metric.upper().replace('_', ' ')}:\n")
                    f.write(f"   Premier: {first_value:.4f}\n")
                    f.write(f"   Dernier: {last_value:.4f}\n")
                    f.write(f"   Évolution: {change:+.2f}% {trend}\n")
            
            # Recommandations
            f.write("\n3. RECOMMANDATIONS\n")
            f.write("-" * 80 + "\n")
            
            latest_metrics = self.metrics_history[-1]
            
            if latest_metrics['ctr'] < 0.10:
                f.write("\n⚠️ CTR faible:\n")
                f.write("   - Améliorer la pertinence des recommandations\n")
                f.write("   - Tester différents algorithmes de ranking\n")
                f.write("   - Analyser les recettes les plus cliquées\n")
            
            if latest_metrics['diversity_score'] < 0.5:
                f.write("\n⚠️ Diversité insuffisante:\n")
                f.write("   - Augmenter la variété des types de recettes\n")
                f.write("   - Équilibrer les recommandations par catégorie\n")
                f.write("   - Introduire plus de découverte (exploration)\n")
            
            if latest_metrics['coverage'] < 0.30:
                f.write("\n⚠️ Couverture limitée:\n")
                f.write("   - Certaines recettes ne sont jamais recommandées\n")
                f.write("   - Implémenter un système de rotation\n")
                f.write("   - Promouvoir les recettes sous-exposées\n")
            
            f.write("\n" + "=" * 80 + "\n")
            f.write("Fin du rapport\n")
            f.write("=" * 80 + "\n")
        
        print(f"✅ Rapport généré: {report_path}")
    
    def visualize_metrics(self):
        """Visualise l'évolution des métriques"""
        print("\n📊 GÉNÉRATION DES GRAPHIQUES")
        
        if len(self.metrics_history) < 2:
            print("⚠️ Pas assez de données pour la visualisation")
            return
        
        df = pd.DataFrame(self.metrics_history)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Créer les graphiques
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Évolution des Métriques de Recommandations', fontsize=16, fontweight='bold')
        
        # CTR
        axes[0, 0].plot(df['timestamp'], df['ctr'], marker='o', color='blue')
        axes[0, 0].set_title('Click-Through Rate (CTR)')
        axes[0, 0].set_ylabel('CTR')
        axes[0, 0].axhline(y=0.10, color='r', linestyle='--', label='Objectif: 10%')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Conversion Rate
        axes[0, 1].plot(df['timestamp'], df['conversion_rate'], marker='o', color='green')
        axes[0, 1].set_title('Taux de Conversion')
        axes[0, 1].set_ylabel('Taux')
        axes[0, 1].axhline(y=0.15, color='r', linestyle='--', label='Objectif: 15%')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Diversity Score
        axes[1, 0].plot(df['timestamp'], df['diversity_score'], marker='o', color='orange')
        axes[1, 0].set_title('Score de Diversité')
        axes[1, 0].set_ylabel('Score (0-1)')
        axes[1, 0].axhline(y=0.70, color='r', linestyle='--', label='Objectif: 0.70')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Average Rating
        axes[1, 1].plot(df['timestamp'], df['avg_rating'], marker='o', color='purple')
        axes[1, 1].set_title('Note Moyenne')
        axes[1, 1].set_ylabel('Note (0-5)')
        axes[1, 1].axhline(y=4.0, color='r', linestyle='--', label='Objectif: 4.0')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('metrics_evolution.png', dpi=300, bbox_inches='tight')
        print("✅ Graphiques sauvegardés: metrics_evolution.png")
        
        plt.close()
    
    def trigger_model_retrain(self):
        """Déclenche le réentraînement du modèle si nécessaire"""
        print("\n🔄 PROCESSUS DE RÉENTRAÎNEMENT")
        print("=" * 80)
        
        # Importer le pipeline de données
        from ai_data_pipeline import AIDataPipeline
        
        pipeline = AIDataPipeline(self.api_base_url)
        
        # Réutiliser le token existant
        pipeline.token = self.token
        
        print("1. Collecte des nouvelles données...")
        data = pipeline.collect_training_data()
        
        print("\n2. Export des données pour l'IA...")
        output_dir = pipeline.export_for_ai_training(data)
        
        print(f"\n✅ Données actualisées dans: {output_dir}/")
        print("   Le modèle peut maintenant utiliser les données fraîches")
        print("\n💡 Note: Gemini utilise les données en temps réel,")
        print("   pas besoin de réentraînement au sens classique.")
    
    def run_full_monitoring_cycle(self):
        """Exécute un cycle complet de monitoring"""
        print("\n" + "=" * 80)
        print("CYCLE COMPLET DE MONITORING")
        print("=" * 80)
        
        # 1. Monitoring des métriques
        metrics = self.monitor_recommendation_quality()
        
        if not metrics:
            print("\n❌ Impossible de continuer sans métriques")
            return
        
        # 2. Génération du rapport
        self.generate_performance_report()
        
        # 3. Visualisation
        try:
            self.visualize_metrics()
        except Exception as e:
            print(f"⚠️ Erreur visualisation: {e}")
        
        # 4. Décision de réentraînement
        health = self._evaluate_system_health(metrics)
        
        if health == 'CRITICAL':
            print("\n🚨 Santé critique détectée")
            response = input("Lancer le réentraînement automatique? (o/n): ")
            if response.lower() == 'o':
                self.trigger_model_retrain()
        
        print("\n" + "=" * 80)
        print("✅ CYCLE DE MONITORING TERMINÉ")
        print("=" * 80)


# Script d'exécution
if __name__ == "__main__":
    print("=" * 80)
    print("SYSTÈME DE MONITORING ET AMÉLIORATION CONTINUE - IA")
    print("=" * 80)
    
    # Configuration
    API_URL = input("\nURL de l'API (défaut: http://localhost:8080/api): ").strip()
    if not API_URL:
        API_URL = "http://localhost:8080/api"
    
    EMAIL = input("Email admin: ").strip()
    PASSWORD = input("Mot de passe: ").strip()
    
    # Initialiser le monitoring
    monitor = AIModelMonitor(api_base_url=API_URL)
    
    # Authentification
    if not monitor.login(EMAIL, PASSWORD):
        print("\n❌ Impossible de continuer sans authentification")
        exit(1)
    
    # Menu principal
    while True:
        print("\n" + "=" * 80)
        print("MENU PRINCIPAL")
        print("=" * 80)
        print("1. Monitoring de la qualité")
        print("2. Générer rapport de performance")
        print("3. Visualiser les métriques")
        print("4. Déclencher réentraînement")
        print("5. Cycle complet de monitoring")
        print("0. Quitter")
        
        choice = input("\nChoisissez une option: ").strip()
        
        if choice == '1':
            monitor.monitor_recommendation_quality()
        elif choice == '2':
            monitor.generate_performance_report()
        elif choice == '3':
            monitor.visualize_metrics()
        elif choice == '4':
            monitor.trigger_model_retrain()
        elif choice == '5':
            monitor.run_full_monitoring_cycle()
        elif choice == '0':
            print("\n👋 Au revoir!")
            break
        else:
            print("❌ Option invalide")